# Forward Kinematics Notebook

This notebook demonstrates forward kinematics for robotic manipulators, covering concepts from Chapter 4: Forward and Inverse Kinematics.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.animation as animation

%matplotlib inline
plt.style.use('seaborn-v0_8')

## 1. Introduction to Forward Kinematics

Forward kinematics is the process of calculating the position and orientation of the end-effector based on the given joint angles. For a robotic manipulator, this involves transforming coordinates from the joint space to the Cartesian space.

In [ ]:
def create_rotation_z_matrix(angle_rad):
    """Create a 3D rotation matrix around the Z-axis."""
    c = np.cos(angle_rad)
    s = np.sin(angle_rad)
    return np.array([
        [c, -s, 0],
        [s, c, 0],
        [0, 0, 1]
    ])

def create_translation_matrix(tx, ty, tz):
    """Create a 4x4 homogeneous transformation matrix with translation."""
    return np.array([
        [1, 0, 0, tx],
        [0, 1, 0, ty],
        [0, 0, 1, tz],
        [0, 0, 0, 1]
    ])

def create_rotation_translation_matrix(rotation_matrix, translation):
    """Create a 4x4 homogeneous transformation matrix from rotation and translation."""
    transform = np.eye(4)
    transform[0:3, 0:3] = rotation_matrix
    transform[0:3, 3] = translation
    return transform

def transform_point(point, transform_matrix):
    """Transform a 3D point using a 4x4 transformation matrix."""
    point_h = np.append(point, 1)  # Convert to homogeneous coordinates
    transformed_h = transform_matrix @ point_h
    return transformed_h[:3]  # Convert back to 3D

def dh_transform(a, alpha, d, theta):
    """
    Create a transformation matrix using Denavit-Hartenberg parameters.
    
    Args:
        a: Link length
        alpha: Link twist
        d: Link offset
        theta: Joint angle
    """
    ct = np.cos(theta)
    st = np.sin(theta)
    ca = np.cos(alpha)
    sa = np.sin(alpha)
    
    return np.array([
        [ct, -st*ca, st*sa, a*ct],
        [st, ct*ca, -ct*sa, a*st],
        [0, sa, ca, d],
        [0, 0, 0, 1]
    ])

## 2. 2-DOF Planar Manipulator

Let's start with a simple 2-DOF planar manipulator to understand the basic concepts.

In [ ]:
class TwoDOFManipulator:
    """
    A class to represent a 2-DOF planar manipulator.
    """
    def __init__(self, link_lengths):
        """
        Initialize the manipulator.
        
        Args:
            link_lengths: List of link lengths [L1, L2]
        """
        self.L1 = link_lengths[0]  # Length of first link
        self.L2 = link_lengths[1]  # Length of second link
    
    def forward_kinematics(self, theta1, theta2):
        """
        Calculate the end-effector position given joint angles.
        
        Args:
            theta1: Angle of first joint (in radians)
            theta2: Angle of second joint (in radians)
        
        Returns:
            (x, y) position of end-effector
        """
        # Calculate position of joint 2
        x1 = self.L1 * np.cos(theta1)
        y1 = self.L1 * np.sin(theta1)
        
        # Calculate position of end-effector
        x2 = x1 + self.L2 * np.cos(theta1 + theta2)
        y2 = y1 + self.L2 * np.sin(theta1 + theta2)
        
        return x2, y2
    
    def forward_kinematics_with_joints(self, theta1, theta2):
        """
        Calculate the end-effector position and intermediate joint positions.
        
        Args:
            theta1: Angle of first joint (in radians)
            theta2: Angle of second joint (in radians)
        
        Returns:
            Dictionary with joint positions
        """
        # Calculate positions
        x1 = self.L1 * np.cos(theta1)
        y1 = self.L1 * np.sin(theta1)
        
        x2 = x1 + self.L2 * np.cos(theta1 + theta2)
        y2 = y1 + self.L2 * np.sin(theta1 + theta2)
        
        return {
            'base': (0, 0),
            'joint1': (x1, y1),
            'end_effector': (x2, y2)
        }
    
    def plot_configuration(self, theta1, theta2, title="2-DOF Manipulator Configuration"):
        """Plot the manipulator configuration at given joint angles."""
        positions = self.forward_kinematics_with_joints(theta1, theta2)
        
        # Extract joint positions
        base = positions['base']
        joint1 = positions['joint1']
        end_effector = positions['end_effector']
        
        # Create the plot
        fig, ax = plt.subplots(figsize=(8, 8))
        
        # Draw links
        ax.plot([base[0], joint1[0]], [base[1], joint1[1]], 'b-', linewidth=6, label=f'Link 1 (L={self.L1})', alpha=0.7)
        ax.plot([joint1[0], end_effector[0]], [joint1[1], end_effector[1]], 'r-', linewidth=6, label=f'Link 2 (L={self.L2})', alpha=0.7)
        
        # Draw joints and end effector
        ax.plot(base[0], base[1], 'ko', markersize=12, label='Base', zorder=5)
        ax.plot(joint1[0], joint1[1], 'bo', markersize=10, label='Joint 1', zorder=5)
        ax.plot(end_effector[0], end_effector[1], 'ro', markersize=10, label='End Effector', zorder=5)
        
        # Mark the end-effector position
        ax.annotate(f'({end_effector[0]:.2f}, {end_effector[1]:.2f})', 
                 xy=end_effector, xytext=(10, 10), 
                 textcoords='offset points', fontsize=10,
                 bbox=dict(boxstyle='round,pad=0.3', facecolor='red', alpha=0.5))
        
        # Set axis properties
        ax.axis('equal')
        ax.grid(True, alpha=0.3)
        ax.set_xlim(-self.L1-self.L2-0.5, self.L1+self.L2+0.5)
        ax.set_ylim(-self.L1-self.L2-0.5, self.L1+self.L2+0.5)
        ax.axhline(0, color='black', linewidth=0.5)
        ax.axvline(0, color='black', linewidth=0.5)
        ax.set_title(title)
        ax.set_xlabel('X position')
        ax.set_ylabel('Y position')
        ax.legend()
        
        plt.show()
        
        # Print details
        print(f"Configuration: θ1={theta1:.3f} rad ({np.degrees(theta1):.1f}°), θ2={theta2:.3f} rad ({np.degrees(theta2):.1f}°)")
        print(f"End-effector position: ({end_effector[0]:.3f}, {end_effector[1]:.3f})")

# Example usage
manipulator = TwoDOFManipulator([1.0, 0.8])
print("2-DOF Manipulator Example")
print("Link lengths: L1=1.0, L2=0.8")
print()

In [ ]:
# Test different configurations
configs = [
    (0, 0, "Both joints at 0 (fully extended horizontally)"),
    (np.pi/2, 0, "First joint at 90 degrees"),
    (0, np.pi/2, "Second joint at 90 degrees"),
    (np.pi/4, np.pi/4, "Both joints at 45 degrees"),
    (np.pi/3, -np.pi/6, "First joint 60 degrees, second joint -30 degrees")
]

for theta1, theta2, title in configs:
    print(f"Configuration: {title}")
    print(f"Joint angles: θ1={theta1:.3f} rad, θ2={theta2:.3f} rad")
    x, y = manipulator.forward_kinematics(theta1, theta2)
    print(f"End-effector position: ({x:.3f}, {y:.3f})")
    print()

## 3. 3-DOF Spatial Manipulator

Now let's look at a more complex 3-DOF spatial manipulator with rotational joints.

In [ ]:
class ThreeDOFSpatial:
    """
    A class to represent a 3-DOF spatial manipulator.
    This is a simplified model for demonstration purposes.
    """
    def __init__(self, link_lengths):
        """
        Initialize the manipulator.
        
        Args:
            link_lengths: List of link lengths [L1, L2, L3]
        """
        self.L1 = link_lengths[0]
        self.L2 = link_lengths[1]
        self.L3 = link_lengths[2]
    
    def forward_kinematics(self, theta1, theta2, theta3):
        """
        Calculate end-effector position for 3-DOF manipulator.
        
        Args:
            theta1: Base rotation (around Z-axis)
            theta2: Shoulder joint (around Y-axis)
            theta3: Elbow joint (around Y-axis)
            
        Returns:
            (x, y, z) position of end-effector
        """
        # Calculate position of each joint
        # Joint 1 (shoulder) - rotated around z-axis
        j1_x = 0
        j1_y = 0
        j1_z = self.L1
        
        # Joint 2 (elbow) - rotated around y-axis
        rot1 = np.array([
            [np.cos(theta1), -np.sin(theta1), 0],
            [np.sin(theta1),  np.cos(theta1), 0],
            [0,               0,              1]
        ])
        
        # Apply second rotation around local y-axis
        link2_dir = np.array([
            self.L2 * np.cos(theta2),  # In local X direction after first rotation
            0,                         # No local Y displacement
            self.L2 * np.sin(theta2)   # In local Z direction
        ])
        
        # Rotate the link2 direction by first joint rotation
        rotated_link2 = rot1 @ link2_dir
        
        j2_x = j1_x + rotated_link2[0]
        j2_y = j1_y + rotated_link2[1]
        j2_z = j1_z + rotated_link2[2]
        
        # Apply third rotation around local y-axis
        link3_dir = np.array([
            self.L3 * np.cos(theta2 + theta3),  # In local X direction
            0,                                  # No local Y displacement
            self.L3 * np.sin(theta2 + theta3)   # In local Z direction
        ])
        
        # The link3 direction is also affected by the first rotation
        rotated_link3 = rot1 @ link3_dir
        
        ee_x = j2_x + rotated_link3[0]
        ee_y = j2_y + rotated_link3[1]
        ee_z = j2_z + rotated_link3[2]
        
        return (ee_x, ee_y, ee_z)
    
    def plot_3d_configuration(self, theta1, theta2, theta3, title="3-DOF Spatial Manipulator"):
        """Create a 3D plot of the manipulator configuration."""
        # Calculate all positions
        ee_pos = self.forward_kinematics(theta1, theta2, theta3)
        
        # For visualization, we need all intermediate points
        # This is a simplified 3D model
        rot1 = np.array([
            [np.cos(theta1), -np.sin(theta1), 0],
            [np.sin(theta1),  np.cos(theta1), 0],
            [0,               0,              1]
        ])
        
        # Shoulder joint (relative to base)
        shoulder_local = np.array([0, 0, self.L1])
        shoulder = rot1 @ shoulder_local
        
        # Elbow joint (relative to shoulder)
        elbow_rel = np.array([self.L2 * np.cos(theta2), 0, self.L2 * np.sin(theta2)])
        elbow = shoulder + rot1 @ elbow_rel
        
        # End-effector (relative to elbow)
        ee_rel = np.array([self.L3 * np.cos(theta2 + theta3), 0, self.L3 * np.sin(theta2 + theta3)])
        ee = elbow + rot1 @ ee_rel
        
        # Create 3D plot
        fig = plt.figure(figsize=(10, 8))
        ax = fig.add_subplot(111, projection='3d')
        
        # Define the path of the manipulator
        x_vals = [0, shoulder[0], elbow[0], ee[0]]
        y_vals = [0, shoulder[1], elbow[1], ee[1]]
        z_vals = [0, shoulder[2], elbow[2], ee[2]]
        
        # Plot the links
        ax.plot(x_vals, y_vals, z_vals, 'b-', linewidth=5, label='Manipulator links')
        
        # Plot the joints
        ax.scatter([0], [0], [0], color='black', s=100, label='Base')
        ax.scatter([shoulder[0]], [shoulder[1]], [shoulder[2]], color='blue', s=100, label='Shoulder')
        ax.scatter([elbow[0]], [elbow[1]], [elbow[2]], color='green', s=100, label='Elbow')
        ax.scatter([ee[0]], [ee[1]], [ee[2]], color='red', s=100, label='End Effector')
        
        # Add text annotations
        ax.text(0, 0, 0, "  Base", None)
        ax.text(shoulder[0], shoulder[1], shoulder[2], "  Shoulder", None)
        ax.text(elbow[0], elbow[1], elbow[2], "  Elbow", None)
        ax.text(ee[0], ee[1], ee[2], "  End Eff", None)
        
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_zlabel('Z')
        ax.set_title(title)
        ax.legend()
        
        plt.show()
        
        print(f"Configuration: θ1={theta1:.3f}, θ2={theta2:.3f}, θ3={theta3:.3f}")
        print(f"End-effector position: ({ee_pos[0]:.3f}, {ee_pos[1]:.3f}, {ee_pos[2]:.3f})")

# Example with 3-DOF manipulator
spatial_manipulator = ThreeDOFSpatial([0.3, 0.5, 0.4])
print("3-DOF Spatial Manipulator Example")
print("Link lengths: L1=0.3, L2=0.5, L3=0.4")
print()

In [ ]:
# Example calculation
theta1, theta2, theta3 = np.pi/4, np.pi/6, -np.pi/6  # 45°, 30°, -30°
ee_pos_3d = spatial_manipulator.forward_kinematics(theta1, theta2, theta3)
print(f"Example: θ1={theta1:.3f}, θ2={theta2:.3f}, θ3={theta3:.3f}")
print(f"End-effector: ({ee_pos_3d[0]:.3f}, {ee_pos_3d[1]:.3f}, {ee_pos_3d[2]:.3f})")

# Visualize if possible
spatial_manipulator.plot_3d_configuration(theta1, theta2, theta3)

## 4. Workspace Analysis

Understanding the workspace of a manipulator is crucial for determining if a particular task is achievable.

In [ ]:
def analyze_workspace_2dof(manipulator, n_samples=1000):
    """
    Analyze and plot the workspace of a 2-DOF manipulator.
    """
    # Generate random joint angles
    theta1_vals = np.random.uniform(-np.pi, np.pi, n_samples)
    theta2_vals = np.random.uniform(-np.pi, np.pi, n_samples)
    
    # Calculate end-effector positions
    end_effector_positions = []
    for t1, t2 in zip(theta1_vals, theta2_vals):
        x, y = manipulator.forward_kinematics(t1, t2)
        end_effector_positions.append((x, y))
    
    # Extract x and y coordinates
    x_coords, y_coords = zip(*end_effector_positions)
    
    # Create the plot
    plt.figure(figsize=(10, 8))
    
    # Plot all positions
    plt.scatter(x_coords, y_coords, s=1, alpha=0.5, label='Reachable positions', color='blue')

    # Draw circles for max and min reach
    angles_plot = np.linspace(0, 2*np.pi, 100)
    max_reach = manipulator.L1 + manipulator.L2
    min_reach = abs(manipulator.L1 - manipulator.L2)

    outer_circle_x = max_reach * np.cos(angles_plot)
    outer_circle_y = max_reach * np.sin(angles_plot)
    plt.plot(outer_circle_x, outer_circle_y, 'r-', linewidth=2, label=f'Max reach ({max_reach})')

    if min_reach > 0:
        inner_circle_x = min_reach * np.cos(angles_plot)
        inner_circle_y = min_reach * np.sin(angles_plot)
        plt.plot(inner_circle_x, inner_circle_y, 'r-', linewidth=2, label=f'Min reach ({min_reach})')
    
    plt.axis('equal')
    plt.grid(True, alpha=0.3)
    plt.title('Workspace Analysis of 2-DOF Manipulator')
    plt.xlabel('X position')
    plt.ylabel('Y position')
    plt.axhline(0, color='black', linewidth=0.5)
    plt.axvline(0, color='black', linewidth=0.5)
    plt.legend()
    plt.show()

    # Calculate workspace area
    if min_reach > 0:
        theoretical_area = np.pi * (max_reach**2 - min_reach**2)
    else:
        theoretical_area = np.pi * max_reach**2
    
    print(f"Robot: L1={manipulator.L1}, L2={manipulator.L2}")
    print(f"Max reach: {max_reach}")
    print(f"Min reach: {min_reach}")
    print(f"Theoretical workspace area: {theoretical_area:.3f} square units")
    print(f"Sampled {n_samples} random configurations")

# Analyze the workspace of our manipulator
analyze_workspace_2dof(manipulator, n_samples=2000)

## 5. Jacobian Matrix

The Jacobian matrix relates joint velocities to end-effector velocities. For a 2-DOF planar manipulator:

In [ ]:
def jacobian_2dof_planar(theta1, theta2, l1, l2):
    """
    Calculate the Jacobian matrix for a 2-DOF planar manipulator.
    
    Args:
        theta1: Angle of first joint
        theta2: Angle of second joint
        l1: Length of first link
        l2: Length of second link
    
    Returns:
        2x2 Jacobian matrix
    """
    # The Jacobian for a 2-DOF planar manipulator is 2x2
    # J = [dx/dt1, dx/dt2]
    #     [dy/dt1, dy/dt2]
    
    # Where x = l1*cos(t1) + l2*cos(t1+t2)
    #       y = l1*sin(t1) + l2*sin(t1+t2)
    
    jacobian = np.zeros((2, 2))
    
    # dx/dt1 = -l1*sin(t1) - l2*sin(t1+t2)
    # dx/dt2 = -l2*sin(t1+t2)
    # dy/dt1 = l1*cos(t1) + l2*cos(t1+t2)
    # dy/dt2 = l2*cos(t1+t2)
    
    jacobian[0, 0] = -l1*np.sin(theta1) - l2*np.sin(theta1 + theta2)  # dx/dtheta1
    jacobian[0, 1] = -l2*np.sin(theta1 + theta2)                       # dx/dtheta2
    jacobian[1, 0] = l1*np.cos(theta1) + l2*np.cos(theta1 + theta2)    # dy/dtheta1
    jacobian[1, 1] = l2*np.cos(theta1 + theta2)                        # dy/dtheta2
    
    return jacobian

# Example: Calculate Jacobian for specific configuration
theta1, theta2 = np.pi/4, np.pi/3
J = jacobian_2dof_planar(theta1, theta2, manipulator.L1, manipulator.L2)

print(f"Configuration: θ1={theta1:.3f}, θ2={theta2:.3f}")
print(f"Jacobian matrix:")
print(J)

# Example use: If joints move at certain velocities, what's the end-effector velocity?
joint_velocities = np.array([0.1, 0.2])  # [theta1_dot, theta2_dot] in rad/s
end_effector_velocity = J @ joint_velocities

print(f"\nIf joint velocities are: θ1_dot={joint_velocities[0]:.2f}, θ2_dot={joint_velocities[1]:.2f}")
print(f"Then end-effector velocity is: dx_dt={end_effector_velocity[0]:.3f}, dy_dt={end_effector_velocity[1]:.3f}")
print(f"Magnitude of end-effector velocity: {np.linalg.norm(end_effector_velocity):.3f}")

## 6. Singularity Analysis

Singularities occur when the Jacobian matrix loses rank, meaning the manipulator loses one or more degrees of freedom.

In [ ]:
def find_singularity_2dof(l1, l2, n_samples=1000):
    """
    Find configurations where the 2-DOF manipulator is in singularity.
    Singularities occur when the determinant of the Jacobian is zero.
    """
    # Sample different configurations
    theta1_vals = np.linspace(-np.pi, np.pi, int(np.sqrt(n_samples)))
    theta2_vals = np.linspace(-np.pi, np.pi, int(np.sqrt(n_samples)))
    
    singular_configs = []
    determinants = []
    
    for t1 in theta1_vals:
        for t2 in theta2_vals:
            J = jacobian_2dof_planar(t1, t2, l1, l2)
            det = np.linalg.det(J)
            determinants.append(abs(det))
            
            # Consider it a singularity if determinant is close to zero
            if abs(det) < 1e-3:
                singular_configs.append((t1, t2))

    print(f"Found {len(singular_configs)} singular configurations in {n_samples} samples")
    
    # For 2-DOF planar manipulator, singularities occur when sin(t2) = 0
    # This happens when t2 = 0 (fully extended) or t2 = π (fully folded)
    print("\nNote: For a 2-DOF manipulator, singularities typically occur when:")
    print("  - The arm is fully stretched (θ2 = 0)")
    print("  - The arm is completely folded (θ2 = π or -π)")
    
    # Show some specific singular configurations
    if singular_configs:
        print(f"\nFirst few singular configurations:")
        for i, (t1, t2) in enumerate(singular_configs[:5]):
            print(f"  Config {i+1}: θ1={t1:.3f}, θ2={t2:.3f}")
            
            # Verify with forward kinematics
            x, y = manipulator.forward_kinematics(t1, t2)
            print(f"             End-effector at: ({x:.3f}, {y:.3f})")
            
            # Calculate Jacobian and its determinant
            J_check = jacobian_2dof_planar(t1, t2, l1, l2)
            det_check = np.linalg.det(J_check)
            print(f"             Jacobian determinant: {det_check:.6f}\n")

# Analyze singularities for our manipulator
find_singularity_2dof(manipulator.L1, manipulator.L2)

## 7. Real-World Application: Path Following

Let's implement a simple simulation where the manipulator follows a predefined path.

In [ ]:
def create_path(n_points=100, path_type="circle"):
    """
    Create a predefined path for the end-effector to follow.
    
    Args:
        n_points: Number of points in the path
        path_type: Type of path ('circle', 'square', 'line')
    """
    if path_type == "circle":
        angles = np.linspace(0, 2*np.pi, n_points)
        center_x, center_y = 0.8, 0.0
        radius = 0.3
        x_path = center_x + radius * np.cos(angles)
        y_path = center_y + radius * np.sin(angles)
    elif path_type == "square":
        # Create a square path
        side_length = 0.4
        quarter_points = n_points // 4
        
        x_parts = []
        y_parts = []
        
        # Bottom side (left to right)
        x_parts.extend(np.linspace(-side_length/2, side_length/2, quarter_points))
        y_parts.extend([0] * quarter_points)
        
        # Right side (bottom to top)
        x_parts.extend([side_length/2] * quarter_points)
        y_parts.extend(np.linspace(0, side_length, quarter_points))
        
        # Top side (right to left)
        x_parts.extend(np.linspace(side_length/2, -side_length/2, quarter_points))
        y_parts.extend([side_length] * quarter_points)
        
        # Left side (top to bottom)
        x_parts.extend([-side_length/2] * (n_points - len(x_parts)))
        y_parts.extend(np.linspace(side_length, 0, n_points - len(y_parts)))
        
        x_path = np.array(x_parts) + 0.8  # Offset to the right
        y_path = np.array(y_parts)
    else:  # line
        x_path = np.linspace(0.3, 1.2, n_points)
        y_path = np.full(n_points, 0.2)  # Horizontal line
    
    return x_path, y_path

def visualize_path_following(manipulator, path_x, path_y, title="Path Following with 2-DOF Manipulator"):
    """Visualize the manipulator following a path."""
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # Plot the desired path
    ax.plot(path_x, path_y, 'g--', linewidth=2, label='Desired Path', alpha=0.7)
    
    # Sample points along the path to show manipulator configurations
    n_samples = min(len(path_x), 20)  # Don't overcrowd the visualization
    indices = np.linspace(0, len(path_x)-1, n_samples, dtype=int)
    
    for idx in indices:
        # For demonstration, use inverse kinematics approach (which we'll approximate)
        # For a more complete solution, implement numerical inverse kinematics
        target_x, target_y = path_x[idx], path_y[idx]
        
        # Simplified approach: just plot the manipulator at each point
        # In reality, we would need to solve inverse kinematics to get joint angles
        # For now, we'll just show the end-effector position on the path
        ax.plot(target_x, target_y, 'ro', markersize=6, alpha=0.6)
    
    ax.axis('equal')
    ax.grid(True, alpha=0.3)
    ax.set_xlim(-manipulator.L1-manipulator.L2-0.5, manipulator.L1+manipulator.L2+0.5)
    ax.set_ylim(-manipulator.L1-manipulator.L2-0.5, manipulator.L1+manipulator.L2+0.5)
    ax.axhline(0, color='black', linewidth=0.5)
    ax.axvline(0, color='black', linewidth=0.5)
    ax.set_title(title)
    ax.set_xlabel('X position')
    ax.set_ylabel('Y position')
    ax.legend()
    
    plt.show()

# Create a circular path
path_x, path_y = create_path(n_points=100, path_type="circle")
visualize_path_following(manipulator, path_x, path_y, "Circular Path Following with 2-DOF Manipulator")

## 8. Summary and Key Concepts

In this notebook, we covered:

1. **Forward Kinematics**: How to calculate end-effector position from joint angles
2. **Transformation Matrices**: Using rotation and translation matrices
3. **Denavit-Hartenberg Convention**: Systematic way to represent robot linkages
4. **Jacobian Matrix**: Relating joint velocities to end-effector velocities
5. **Workspace Analysis**: Understanding the reachable space of the manipulator
6. **Singularity Analysis**: Identifying configurations with reduced mobility
7. **Path Following**: Conceptual approach to trajectory following

Forward kinematics is the foundation for more complex robotic operations like inverse kinematics, motion planning, and control. Understanding these concepts is crucial for developing robotic applications.

In [ ]:
# Final summary
print("SUMMARY OF FORWARD KINEMATICS")
print("=" * 40)
print("1. Forward kinematics calculates end-effector pose from joint angles")
print("2. Transformation matrices combine rotation and translation")
print("3. The Jacobian relates joint velocities to end-effector velocities")
print("4. Workspace analysis determines reachable positions")
print("5. Singularities occur when the Jacobian loses rank")
print("6. These concepts form the basis for motion planning and control")